In [ ]:
import geopandas as gpd
import pandas as pd

# Part 3 - Warsaw population analysis and visualization

This part will utilize geospatial knowledge presented earlier to prepare a visualization of Warsaw population distribution

### Task 1

Load Warsaw census data and districts boundaries. At the end find the top 3 most populated districts and plot their boundaries on the map

Data files:
* `../../data/warsaw_population.json`
* `../../data/warsaw_districts.geojson`

Remember to set CRS when loading GeoDataFrames. For most cases, the best choice is `WGS84/EPSG:4326` (same crs, two different names). You can set it using `to_crs()` function, for example `gdf.to_crs(epsg=4326)`

In [ ]:
warsaw_population = ...
warsaw_districts = ...

# YOUR CODE HERE
raise NotImplementedError()

### Task 2

Load all buildings in Warsaw. You can use `OSMOnlineLoader` from the `srai` library. 

In [ ]:
from srai.loaders.osm_loaders import OSMOnlineLoader

loader = OSMOnlineLoader()

osm_building_types = [
    "residential",
    "apartments",
    "house",
    "semidetached_house",
    "detached",
]

osm_filter = {
    "building": osm_building_types,
    "building:levels": True,
}

In [ ]:
warsaw_polygon = ...  # merge all districts into one polygon
buildings = ... # and load osm data for this polygon

# YOUR CODE HERE
raise NotImplementedError()

buildings.head()

OSM loader looks at those two tags (*building* and *building:levels*) independently. We need to clean the result to leave only entries with both of those tags. 

For cleaning we should assume that:
* we skip buildings of unknown type
* buildings without levels are assumed to have 1 floor
* levels should be integers

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

buildings["building:levels"].hist()

### Task 3

Approximate the distribution of population across buildings. We will do this in four steps:

* Calculate *inhabited_area* of each building, which we understand as a multiplication of its area by the number of floors. This is based on an assumption that in taller building lives more people
* Simplify each building to the single point on the map instead of a polygon
* Calculate *total_inhabited_area* for each district
* Calculate population of each building from the equation: `population = district_population * inhabited_area / total_inhabited_area`

TIP: Remember that all geometrical calculations (area, circumference, centroid) requires proper CRS setting (`epsg=2180` is best for Poland). After that, bring the results back to `epsg=4326`

In [ ]:
# Start with first two tasks - caclulate the inhabited_area and convert buildings to points

# YOUR CODE HERE
raise NotImplementedError()

# This is some magic coordinates to zoom on a part of Warsaw
xmin = 21.042753111534097
xmax = 21.069257679735955
ymin = 52.24187245384607
ymax = 52.22533280016626

buildings.cx[xmin:xmax, ymin:ymax].explore()

In [ ]:
# Next, match buildings to districts, using spatial join operation and calculate total_inhabited_area for each district

buildings_with_districts = ...

# YOUR CODE HERE
raise NotImplementedError()

buildings_with_districts.head()

In [ ]:
# Calculate total_inhabited_area for each district

totals_in_districts = ...

# YOUR CODE HERE
raise NotImplementedError()

totals_in_districts.head()

In [ ]:
# Finally, calculate population in each building

buildings_with_population = ...

# YOUR CODE HERE
raise NotImplementedError()

buildings_with_population.head()

### Task 4

Aggregate data to H3 cells. This will allow us to prepare higher quality visualizations. Task to do:

* convert each building (point) to it's corresponding H3 cell ID (cellect appropriate resolution, you can look [here](https://wolf-h3-viewer.glitch.me/) for reference)
* calculate sum of population in each cell
* create a geometry column for each H3 cell

In [ ]:
import h3

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

buildings_with_population["h3"].value_counts()

In [ ]:
h3_population = ...

# YOUR CODE HERE
raise NotImplementedError()

h3_population["population_in_h3"].sum()

In [ ]:
from srai.h3 import h3_to_geoseries

In [ ]:
h3_population_gdf = ...

# YOUR CODE HERE
raise NotImplementedError()

h3_population_gdf.head()

### Task 5

Show data on the map. You can test 2 different approaches to plotting:

* built-in `.explore()` method from `GeoPandas`
* plotting function from `srai.plotting`

At the end we prepared an advanced, 3D map visualization based on Deck.gl

In [ ]:
# base folium - explore


# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# srai - plot numeric
# TIP: the library requires the index with the name "region_id" to be set, use "h3" column for that
from srai.plotting import plot_numeric_data

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# pydeck 3d
from srai.plotting.folium_wrapper import _generate_linear_colormap
import plotly.express as px
import pydeck as pdk

colormap = _generate_linear_colormap(
    # https://plotly.com/python/builtin-colorscales/
    px.colors.sequential.Aggrnyl_r,
    min_value=h3_population_gdf["population_in_h3"].min(),
    max_value=h3_population_gdf["population_in_h3"].max(),
)

h3_population_gdf["color"] = h3_population_gdf["population_in_h3"].map(
    colormap.rgb_bytes_tuple
)

# Define a layer to display on a map
layer = pdk.Layer(
    "H3HexagonLayer",
    h3_population_gdf,
    pickable=True,
    stroked=True,
    filled=True,
    extruded=True,
    get_hexagon="h3",
    get_fill_color="[color[0], color[1], color[2], 204]",
    elevation_scale=0.5,
    get_elevation="population_in_h3",
    coverage=0.8,
)

# Set the viewport location
view_state = pdk.ViewState(
    latitude=52.2317, longitude=21.0062, zoom=9.5, bearing=0, pitch=30
)


# Render
pdk.Deck(
    layers=[layer],
    map_style="light",
    initial_view_state=view_state,
    tooltip={"text": "Population: {population_in_h3}"},
)